# Análise Exploratória de Dados – Job Postings

Este notebook apresenta uma análise exploratória do dataset **fake_real_job_postings**, incluindo:

- Estatísticas descritivas
- Detecção de outliers
- Boxplots
- Distribuições e regressões
- Matriz de correlação

O objetivo é identificar padrões, anomalias e relações relevantes entre as variáveis,
servindo como base para análises estatísticas e modelos preditivos.


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, length, to_date, countDistinct
import plotly.express as px
import pandas as pd


In [2]:
spark = SparkSession.builder \
    .appName("Analise_Raw_Layer") \
    .master("local[*]") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 15:05:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/19 15:05:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_raw = spark.read.csv("data_raw.csv", header=True, inferSchema=True)
print("Dados carregados. Total de linhas:", df_raw.count())

Dados carregados. Total de linhas: 3000


## 1. Análise de Completude dos Dados

### 1.1 Completude Global das Colunas
Esta análise avalia o percentual de valores preenchidos em cada coluna do dataset.
O objetivo é identificar campos com alto volume de dados ausentes, que podem comprometer
análises estatísticas e modelagens futuras.

In [4]:
total_rows = df_raw.count()

completeness_exprs = []

for c in df_raw.columns:

    expressao = count(when(col(c).isNotNull(), c))

    percentual = (expressao / total_rows * 100).alias(c)
    completeness_exprs.append(percentual)

quality_df = df_raw.select(*completeness_exprs).toPandas().transpose().reset_index()
quality_df.columns = ['Coluna', 'Preenchimento']
quality_df = quality_df.sort_values('Preenchimento')

fig1 = px.bar(quality_df, x='Preenchimento', y='Coluna', orientation='h',
              title='1. Completude: Percentual de Dados Presentes',
              color='Preenchimento', color_continuous_scale='RdYlGn', text_auto='.1f')
fig1.update_layout(height=600)
fig1.show()

## 2. Análise de Cardinalidade

### 2.1 Distribuição de Valores Únicos
A cardinalidade representa a quantidade de valores distintos em cada coluna.
Esse indicador auxilia na identificação de campos categóricos, identificadores
e atributos de alta variabilidade.

In [5]:
card_df = df_raw.select([countDistinct(c).alias(c) for c in df_raw.columns]).toPandas().transpose()
card_df.columns = ['Valores Unicos']
card_df = card_df.sort_values('Valores Unicos', ascending=False).head(15)

fig2 = px.bar(card_df, x='Valores Unicos', y=card_df.index, orientation='h',
              title='2. Cardinalidade: Top 15 Colunas com Maior Variedade',
              text_auto=True)
fig2.update_xaxes(type="log", title="Qtd. Valores Únicos (Escala Log)")
fig2.show()

26/01/19 15:05:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 3. Perfil Técnico dos Dados

### 3.1 Análise de Metadados e Tipagem
Esta análise apresenta os tipos de dados inferidos durante a ingestão.
Seu objetivo é validar a coerência técnica dos campos e identificar possíveis
inconsistências de tipagem.

In [6]:
dtypes_df = pd.DataFrame(df_raw.dtypes, columns=['Coluna', 'Tipo'])
type_counts = dtypes_df['Tipo'].value_counts().reset_index()
type_counts.columns = ['Tipo', 'Contagem']

fig3 = px.pie(type_counts, values='Contagem', names='Tipo',
              title='3. Perfil Técnico: Tipos de Dados Ingeridos')
fig3.show()

## 4. Análise de Valores Ausentes

### 4.1 Quantificação de Dados Faltantes
Esta etapa identifica colunas que apresentam valores nulos e quantifica
seu impacto relativo no conjunto de dados.

In [7]:
total_rows = df_raw.count()

null_exprs = [count(when(col(c).isNull(), c)).alias(c) for c in df_raw.columns]
null_counts = df_raw.select(null_exprs).collect()[0].asDict()

data_list = []
for coluna, nulos in null_counts.items():
    if nulos > 0:
        percentual = (nulos / total_rows) * 100
        label_texto = f"{nulos:,} ({percentual:.1f}%)"

        data_list.append({
            "Coluna": coluna,
            "Nulos": nulos,
            "Total": total_rows,
            "%": percentual,
            "Label": label_texto
        })

df_missing = pd.DataFrame(data_list).sort_values(by="Nulos", ascending=True)

display(df_missing.sort_values(by="Nulos", ascending=False))

,Coluna,Nulos,Total,%,Label
2,fraud_reason,1528,3000,50.933333,"1,528 (50.9%)"
0,company_name,1472,3000,49.066667,"1,472 (49.1%)"
1,company_website,1472,3000,49.066667,"1,472 (49.1%)"


### 4.2 Visualização das Colunas com Maior Ausência
O gráfico destaca as colunas com maior volume absoluto de dados ausentes,
facilitando a priorização de estratégias de tratamento.


In [8]:
df_plot = df_missing.tail(15)

fig = px.bar(
    df_plot,
    x="Nulos",
    y="Coluna",
    orientation='h',
    text="Label",
    title="Top Colunas com Dados Faltantes",
    color="Nulos",
    color_continuous_scale="RdBu_r"
)

fig.update_traces(
    textposition='outside',
    cliponaxis=False
)

fig.update_layout(
    xaxis_title="Quantidade de Valores Nulos",
    yaxis_title=None,
    coloraxis_showscale=False,
    height=500,
    margin=dict(r=100)
)

fig.show()

## 5. Consistência de Domínio

### 5.1 Distribuição por Indústria
Esta análise apresenta as indústrias mais frequentes no dataset, auxiliando
na identificação de concentração e possíveis vieses de domínio.

In [9]:
top_ind = df_raw.groupBy("industry").count().orderBy(col("count").desc()).limit(10).toPandas()

fig6 = px.bar(top_ind, x='count', y='industry', orientation='h',
              title='5. Consistência de Domínio: Top Indústrias', text_auto=True)
fig6.update_layout(yaxis={'categoryorder':'total ascending'})
fig6.show()

## 6. Consistência Geográfica

### 6.1 Distribuição por Localização
Avalia a distribuição das vagas por localidade, permitindo identificar
concentração geográfica e padrões de preenchimento.

In [10]:
top_loc = df_raw.groupBy("location").count().orderBy(col("count").desc()).limit(10).toPandas()

fig7 = px.bar(top_loc, x='count', y='location', orientation='h',
              title='6. Consistência Geográfica: Top Localizações',
              color_discrete_sequence=['teal'], text_auto=True)
fig7.update_layout(yaxis={'categoryorder':'total ascending'})
fig7.show()

## 7. Validação Numérica: Anos de Experiência

Esta análise avalia a consistência do campo *required_experience_years* a partir
da classificação dos valores em faixas de experiência profissional.

A utilização de uma tabela permite identificar registros não informados,
concentração por nível de senioridade e valores atípicos, facilitando a validação
do campo e a definição de estratégias de tratamento para análises futuras.

In [11]:
from pyspark.sql.functions import col, when

total = df_raw.count()

exp_table = (
    df_raw
    .select(col("required_experience_years").cast("int").alias("anos"))
    .withColumn(
        "Faixa de Experiência",
        when(col("anos").isNull(), "Não informado")
        .when(col("anos") == 0, "Não informado")
        .when(col("anos") <= 2, "Júnior (1–2 anos)")
        .when(col("anos") <= 5, "Pleno (3–5 anos)")
        .when(col("anos") <= 10, "Sênior (6–10 anos)")
        .otherwise("Atípico (> 10 anos)")
    )
    .groupBy("Faixa de Experiência")
    .count()
    .withColumn(
        "Percentual (%)",
        (col("count") / total) * 100
    )
    .orderBy("count", ascending=False)
    .toPandas()
)

exp_table.rename(columns={"count": "Quantidade"}, inplace=True)

exp_table


,Faixa de Experiência,Quantidade,Percentual (%)
0,Sênior (6–10 anos),1377,45.900000
1,Pleno (3–5 anos),823,27.433333
2,Júnior (1–2 anos),531,17.700000
3,Não informado,269,8.966667


## 8. Análise Temporal

### 8.1 Distribuição Temporal das Vagas
Avalia o volume de vagas publicadas ao longo do tempo, identificando padrões,
picos de ingestão e possíveis falhas de coleta.

In [12]:
date_dist = df_raw.select(to_date(col("posting_date")).alias("data")) \
                  .groupBy("data").count().orderBy("data").toPandas()

fig8 = px.line(date_dist, x='data', y='count', markers=True,
               title='8. Distribuição Temporal das Vagas')
fig8.show()

## 9. Análise de Flags Binárias

### 9.1 Balanceamento de Variáveis Booleanas
Esta seção analisa o balanceamento das variáveis booleanas do dataset por meio de
gráficos individuais, permitindo uma visualização clara da distribuição de cada
atributo.

A variável *telecommuting* diferencia vagas presenciais e vagas com possibilidade
de trabalho remoto, possibilitando a avaliação do grau de adoção do modelo remoto
no conjunto de dados.

A variável *has_logo* indica a presença ou ausência de logotipo da empresa associada
à vaga, podendo refletir tanto características do anunciante quanto limitações na
coleta de dados.

Por fim, a variável *is_fake* classifica as vagas entre reais e falsas, sendo
fundamental para análises de qualidade dos dados e para futuras aplicações de
modelagem preditiva voltadas à detecção de vagas fraudulentas.

In [13]:
temp = (
    df_raw
    .groupBy("telecommuting")
    .count()
    .toPandas()
)

temp["telecommuting"] = temp["telecommuting"].map({
    0: "Vaga Presencial",
    1: "Vaga Remota"
})

fig_tc = px.bar(
    temp,
    x="telecommuting",
    y="count",
    text="count",
    title="9.1 Distribuição das Vagas por Modalidade de Trabalho (Telecommuting)"
)

fig_tc.update_layout(
    xaxis_title="Modalidade da Vaga",
    yaxis_title="Quantidade de Registros",
    showlegend=False
)

fig_tc.show()


In [14]:
temp = (
    df_raw
    .groupBy("has_logo")
    .count()
    .toPandas()
)

temp["has_logo"] = temp["has_logo"].map({
    0: "Empresa sem Logotipo",
    1: "Empresa com Logotipo"
})

fig_logo = px.bar(
    temp,
    x="has_logo",
    y="count",
    text="count",
    title="9.2 Distribuição das Vagas por Presença de Logotipo da Empresa"
)

fig_logo.update_layout(
    xaxis_title="Informação de Logotipo",
    yaxis_title="Quantidade de Registros",
    showlegend=False
)

fig_logo.show()


In [15]:
temp = (
    df_raw
    .groupBy("is_fake")
    .count()
    .toPandas()
)

temp["is_fake"] = temp["is_fake"].map({
    0: "Vaga Real",
    1: "Vaga Falsa"
})

fig_fake = px.bar(
    temp,
    x="is_fake",
    y="count",
    text="count",
    title="9.3 Distribuição das Vagas por Veracidade (Reais vs Falsas)"
)

fig_fake.update_layout(
    xaxis_title="Classificação da Vaga",
    yaxis_title="Quantidade de Registros",
    showlegend=False
)

fig_fake.show()


In [16]:
fraud_df = df_raw \
    .filter(col("is_fake") == 1) \
    .filter(col("fraud_reason").isNotNull()) \
    .groupBy("fraud_reason") \
    .count() \
    .orderBy(col("count").desc()) \
    .toPandas()

fraud_df = fraud_df.head(10)

fig = px.bar(
    fraud_df,
    x="count",
    y="fraud_reason",
    orientation="h",
    title="Razões Mais Frequentes para Classificação de Vagas Fraudulentas",
    labels={
        "count": "Quantidade de Registros",
        "fraud_reason": "Motivo da Fraude"
    },
    text_auto=True
)

fig.update_layout(
    yaxis={"categoryorder": "total ascending"},
    height=500
)

fig.show()